# ⚡ Day 6 — Facebook Prophet + Ensemble Stacking
## Energy Consumption Forecasting | Claysys AI Hackathon 2026

**Date:** February 24, 2026  
**Objective:** Apply Meta's Prophet model and build an ensemble that combines the best models from previous days.

---

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import warnings
warnings.filterwarnings('ignore')

from src.models.prophet_model import ProphetForecaster
from src.evaluation import compute_metrics, plot_predictions, compare_models, plot_model_comparison

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'figure.dpi': 120})
print('✅ Day 6 Setup complete')

## 1. Load Data

In [ ]:
train_df = pd.read_csv('../data/processed/train.csv', index_col='Datetime', parse_dates=True)
test_df  = pd.read_csv('../data/processed/test.csv',  index_col='Datetime', parse_dates=True)

train_series = train_df['Global_active_power']
test_series  = test_df['Global_active_power']

print(f'Train: {len(train_series):,} hourly records')
print(f'Test : {len(test_series):,} hourly records')

## 2. Facebook Prophet

In [ ]:
# Prophet needs ds + y format, handled inside ProphetForecaster
prophet_model = ProphetForecaster(
    seasonality_mode='additive',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.05,
)

# Fit on training series
prophet_model.fit(train_series)

# Predict for the test horizon
prophet_preds = prophet_model.get_predictions_array(len(test_series), freq='h')
metrics_prophet = compute_metrics(test_series.values, prophet_preds, model_name='Prophet')

prophet_model.plot_forecast(save=True)
prophet_model.plot_components(save=True)

In [ ]:
plot_predictions(test_series.values[:168], prophet_preds[:168],
                 index=test_df.index[:168],
                 model_name='Prophet (First Week of Test)',
                 filename='prophet_predictions.png')

## 3. Ensemble: Weighted Average of Best Models

In [ ]:
# Load ML predictions already generated in Day 4
# (Re-generate them here for the ensemble)
from src.models.ml_models import RandomForestForecaster, XGBoostForecaster, LightGBMForecaster

drop_cols = ['season']
train_ml = train_df.drop(columns=[c for c in drop_cols if c in train_df.columns])
test_ml  = test_df.drop(columns=[c for c in drop_cols if c in test_df.columns])

TARGET = 'Global_active_power'
feature_cols = [c for c in train_ml.select_dtypes(include=[np.number]).columns if c != TARGET]

X_train, y_train = train_ml[feature_cols], train_ml[TARGET]
X_test,  y_test  = test_ml[feature_cols],  test_ml[TARGET]

# Load or re-fit models
xgb_model = XGBoostForecaster(n_estimators=500, learning_rate=0.05)
xgb_model.fit(X_train.iloc[:int(len(X_train)*0.9)], y_train.iloc[:int(len(y_train)*0.9)])
xgb_preds = xgb_model.predict(X_test)

lgbm_model = LightGBMForecaster(n_estimators=500, learning_rate=0.05)
lgbm_model.fit(X_train.iloc[:int(len(X_train)*0.9)], y_train.iloc[:int(len(y_train)*0.9)])
lgbm_preds = lgbm_model.predict(X_test)

print('✅ ML models re-fitted for ensemble.')

In [ ]:
# Weighted Ensemble: XGBoost 40% + LightGBM 40% + Prophet 20%
ensemble_preds = 0.40 * xgb_preds + 0.40 * lgbm_preds + 0.20 * prophet_preds
metrics_ensemble = compute_metrics(y_test.values, ensemble_preds, model_name='Ensemble (XGB+LGBM+Prophet)')

plot_predictions(y_test.values[:168], ensemble_preds[:168],
                 index=test_df.index[:168],
                 model_name='Ensemble (First Week of Test)',
                 filename='ensemble_predictions.png')

## 4. All-Model Comparison (Day 6 Summary)

In [ ]:
# Load previous day metrics
baseline_results = pd.read_csv('../reports/baseline_results.csv', index_col=0)
ml_results       = pd.read_csv('../reports/ml_results.csv', index_col=0)

# Add today's results
new_results = pd.DataFrame([metrics_prophet, metrics_ensemble]).set_index('Model')

# Combine all
all_results = pd.concat([baseline_results, ml_results, new_results])
all_results = all_results.sort_values('RMSE')

print('\n🏆 FULL MODEL LEADERBOARD:')
print(all_results.to_string())

all_results.to_csv('../reports/all_model_results.csv')

plot_model_comparison(all_results, metric='RMSE')
plot_model_comparison(all_results, metric='MAPE')

print('\n🎉 Day 6 Complete! Prophet + Ensemble done.')
print(f'   🥇 Best Model: {all_results.index[0]} (RMSE={all_results["RMSE"].iloc[0]:.4f})')
print('   Ready for Day 7: Final Report & Submission!')